In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

13.1

In [ ]:
file_path = "shapes_rotation/events.txt"
lines = []
try:
    with open(file_path, "r") as file:
        for line in file:
            line = line.replace("\n", "").split(" ")
            if float(line[0]) >= 1:
                break
            lines.append(line)

except FileNotFoundError:
    print(f"File {file_path} not found.")
except Exception as e:
    print(f"An error occurred: {e}")

timestamp, x, y, polarity = [], [], [], []

for t, x1, y1, p in lines:
    timestamp.append(float(t))
    x.append(int(x1))
    y.append(int(y1))
    polarity.append(int(p))



In [ ]:
print("number of events: ", len(timestamp))
print("first timestamp: ", timestamp[0])
print("last timestamp: ", timestamp[-1])
print("max x: ", max(x))
print("max y: ", max(y))
print("min x: ", min(x))
print("min y: ", min(y))

positive_polarity = []
negative_polarity = []

for p in polarity:
    if p == 1:
        positive_polarity.append(p)
    else:
        negative_polarity.append(p)

positive_polarity_nr = len(positive_polarity)
negative_polarity_nr = len(negative_polarity)

print("number of positive polarities: ", positive_polarity_nr)
print("number of negative polarities: ", negative_polarity_nr)



In [ ]:
fig = plt.figure(figsize=(12, 13))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, timestamp, c=polarity, cmap='viridis')
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Timestamp')
ax.set_title('3D Scatter Plot of Events')
#ax.view_init(45,45)
plt.colorbar(ax.collections[0], label='Polarity')
plt.show()

13.2

In [ ]:
fig = plt.figure(figsize=(12, 13))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x[:8000], y[:8000], timestamp[:8000], c=polarity[:8000], cmap='viridis')
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Timestamp')
ax.set_title('3D Scatter Plot of Events')
ax.view_init(270,90)
plt.colorbar(ax.collections[0], label='Polarity')
plt.show()

img = cv2.imread("./shapes_rotation/images/frame_00000000.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(img, cmap='gray')
plt.axis('off')
plt.title('First Image Frame')
plt.show()

In [ ]:
index_05 = np.argmax(np.array(timestamp) > 0.5)

fig = plt.figure(figsize=(12, 13))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x[index_05:], y[index_05:], timestamp[index_05:], c=polarity[index_05:], cmap='viridis')
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Timestamp')
ax.set_title('3D Scatter Plot of Events')
#ax.view_init(270,90)
plt.colorbar(ax.collections[0], label='Polarity')
plt.show()


In [ ]:
#How long is the sequence used during exercise 1.1 (in seconds)?
print("Time of sequence: ", (timestamp[-1] - timestamp[0]), " seconds")

#What’s the resolution of event timestamps in microseconds?
resolution = (timestamp[-1] - timestamp[0]) / len(timestamp)
print("Average resolution of event timestamps: ", resolution * 1e6, " microseconds")

#What does the time difference between consecutive events depend on?
#The time difference between consecutive events depends on the speed of the object being tracked, the frame rate of the camera, and the event-based nature of the sensor.

#What does positive/negative event polarity mean?
#Positive polarity means that the event is triggered by an increase in brightness, while negative polarity means that the event is triggered by a decrease in brightness.

#What is the direction of movement of objects in exercise 1.2?
#Counter-clockwise rotation of the object.


13.3

In [ ]:
file_path = "shapes_rotation/events.txt"
lines = []
try:
    with open(file_path, "r") as file:
        for line in file:
            line = line.replace("\n", "").split(" ")
            if float(line[0]) <= 1:
                continue
            if float(line[0]) >= 2:
                break

            lines.append(line)

except FileNotFoundError:
    print(f"File {file_path} not found.")
except Exception as e:
    print(f"An error occurred: {e}")

timestamp, x, y, polarity = [], [], [], []

for t, x1, y1, p in lines:
    timestamp.append(float(t))
    x.append(int(x1))
    y.append(int(y1))
    polarity.append(1 if int(p) == 1 else -1)


In [ ]:
def event_frame(coords, polarities, img_shape):
    frame = np.zeros(img_shape, dtype=np.uint8) + 127
    frame = frame.astype(np.uint8)

    for i, (x, y) in enumerate(coords):
       if polarities[i] == 1:
           frame[y, x] = 255
       else:
           frame[y, x] = 0

    return frame

In [ ]:
tau = 0.01

In [ ]:
start_index = 0

while start_index < len(timestamp):
    temp_timestamp = []
    temp_coords = []
    temp_polarities = []

    start_time = timestamp[start_index]

    for i in range(start_index, len(timestamp)):
        if timestamp[i] - start_time < tau:
            temp_timestamp.append(timestamp[i])
            temp_coords.append((x[i], y[i]))
            temp_polarities.append(polarity[i])
        else:
            break

    frame = event_frame(temp_coords, temp_polarities, (180, 240))
    cv2.imshow("Event Frame", frame)
    if cv2.waitKey(0) == ord('q'):
       cv2.destroyAllWindows()
       break
    start_index = i

cv2.destroyAllWindows()